In [ ]:
# sacado de la documentación, yo no lo he ejecutado.
# %pip install llama-index-llms-openai llama-index-embeddings-openai

# estos son los módulos que tengo instalados
# pip install llama-index
# pip install llama-index-embeddings-huggingface
# pip install llama-index-llms-ollama
# pip install matplotlib

In [1]:
# attach to the same event-loop
import nest_asyncio

nest_asyncio.apply()

In [2]:
import os
import openai

os.environ["OPENAI_API_KEY"] = "TU KEY"
# openai.api_key = os.environ["OPENAI_API_KEY"]

In [3]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Response
from llama_index.llms.openai import OpenAI
from llama_index.core.evaluation import (
    FaithfulnessEvaluator,
    RelevancyEvaluator,
    CorrectnessEvaluator,
    GuidelineEvaluator,
    SemanticSimilarityEvaluator
)
from llama_index.core.node_parser import SentenceSplitter
import pandas as pd

pd.set_option("display.max_colwidth", 0)

In [4]:
# gpt-4
gpt4 = OpenAI(temperature=0, model="gpt-4")

faithfulness_gpt4 = FaithfulnessEvaluator(llm=gpt4)
relevancy_gpt4 = RelevancyEvaluator(llm=gpt4)
correctness_gpt4 = CorrectnessEvaluator(llm=gpt4)
semantic = SemanticSimilarityEvaluator()  #coge el modelo de embedings de settings si no se le pasa ninguno.

In [5]:
# para evaluar guidelines, poner cada guideline en el array,
# y se crea una lista con un GuidelineEvaluator para cada guideline.

GUIDELINES = [
    "The response should fully answer the query.",
    "The response should avoid being vague or ambiguous.",
    (
        "The response should be specific and use statistics or numbers when"
        " possible."
    ),
]

guideline_evaluators = [
    GuidelineEvaluator(llm=gpt4, guidelines=guideline)
    for guideline in GUIDELINES
]

In [6]:
from llama_index.core.llama_dataset import download_llama_dataset, LabelledRagDataset

rag_dataset = LabelledRagDataset.from_json("./eval_data/rag_data/paul_graham/rag_dataset.json")
documents = SimpleDirectoryReader(input_dir="./eval_data/rag_data/paul_graham/source_files").load_data()

In [7]:
# en el caso de que quisieramos generar un dataset de queries a raíz de los documentos

# from llama_index.core.evaluation import DatasetGenerator

# dataset_generator = DatasetGenerator.from_documents(documents, llm=llm)

# qas = dataset_generator.generate_dataset_from_nodes(num=3)

In [8]:
rag_dataset_pandas = rag_dataset.to_pandas()
queries = rag_dataset_pandas["query"]
reference_answers = rag_dataset_pandas["reference_answer"]

In [9]:
# este paso lo hago simplemente para reducir el tamaño del dataset,
# para que no tarde mucho la evaluación cada vez que hago pruebas durante el desarrollo

queries = queries[:5]
reference_answers = reference_answers[:5].to_list()

In [10]:
print(len(queries))
print(len(reference_answers))

5
5


In [11]:
# aquí crear el modelo que quieras evaluar

# create vector index
llm = OpenAI(temperature=0.3, model="gpt-3.5-turbo")
splitter = SentenceSplitter(chunk_size=512)
vector_index = VectorStoreIndex.from_documents(
    documents, transformations=[splitter]
)

In [79]:
from llama_index.core.evaluation import BatchEvalRunner

# ponemos todos los evaluadores que queremos usar aquí.
# para evaluar los guidelines, poner cada evaluador del array guideline_evaluators en el diccionario, igual que el resto de evaluadores.
runner = BatchEvalRunner(
    {"correctness": correctness_gpt4, 
     "faithfulness": faithfulness_gpt4, 
     "relevancy": relevancy_gpt4, 
     "guideline": guideline_evaluators[0], 
     "semantic": semantic},
    workers=8,
)

# he corregido una cosa de aquí del ejemplo, que no estaba (llm=llm), supongo que por error.
eval_results = await runner.aevaluate_queries(
    vector_index.as_query_engine(llm=llm),
    queries=queries,
    reference=reference_answers,
)

Retrying llama_index.llms.openai.base.OpenAI._achat in 0.9158085201245835 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-bduuTBXywQ8L0sEG7RvWDi3T on tokens per min (TPM): Limit 10000, Used 9771, Requested 1310. Please try again in 6.486s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.5482750352021657 seconds as it raised RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4 in organization org-bduuTBXywQ8L0sEG7RvWDi3T on tokens per min (TPM): Limit 10000, Used 9664, Requested 1418. Please try again in 6.492s. Visit https://platform.openai.com/account/rate-limits to learn more.', 'type': 'tokens', 'param': None, 'code': 'rate_limit_exceeded'}}.
Retrying llama_index.llms.openai.base.OpenAI._achat in 0.06162871811830184 seconds as it

In [81]:
print(eval_results.keys())

print(eval_results["semantic"][0].dict().keys())

print(eval_results["semantic"][0].passing)
print(eval_results["semantic"][0].response)
print(eval_results["semantic"][0].contexts)

dict_keys(['correctness', 'faithfulness', 'relevancy', 'guideline', 'semantic'])
dict_keys(['query', 'contexts', 'response', 'passing', 'feedback', 'score', 'pairwise_source', 'invalid_result', 'invalid_reason'])
True
None
None


In [82]:
# función que venía en el ejemplo de la documentación, pero que he modificado por la de más abajo.
# def get_eval_results(key, eval_results):
#     results = eval_results[key]
#     correct = 0
#     for result in results:
#         if result.passing:
#             correct += 1
#     score = correct / len(results)
#     print(f"{key} Score: {score}")
#     return score

In [84]:
# he modificado la del ejemplo para que use result.score en vez de result.passing. en algunos tipos de evaluación sale un resultado más preciso
def get_eval_results(key, eval_results):
    results = eval_results[key]
    sum_score = 0
    for result in results:
        sum_score += result.score
    score = sum_score / len(results)
    print(f"{key} Score: {score}")
    return score

In [85]:
score = get_eval_results("faithfulness", eval_results)

faithfulness Score: 1.0


In [86]:
score = get_eval_results("relevancy", eval_results)

relevancy Score: 1.0


In [87]:
score = get_eval_results("correctness", eval_results)

correctness Score: 4.6


In [88]:
score = get_eval_results("guideline", eval_results)

guideline Score: 1.0


In [89]:
score = get_eval_results("semantic", eval_results)

semantic Score: 0.9714864542186652


In [90]:
print(eval_results["faithfulness"][:1])

[EvaluationResult(query='In the essay, the author mentions his early experiences with programming. Describe the first computer he used for programming, the language he used, and the challenges he faced.', contexts=["I remember vividly how impressed and envious I felt watching him sitting in front of it, typing programs right into the computer.\n\nComputers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Apple II, but a TRS-80 was good enough. This was when I really started programming. I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and then print them out, but it was a lot better than a typewriter.\n\nThough I liked programming, I didn't plan to study it in college. In college I was going t

In [91]:
print(eval_results["relevancy"][:1])

[EvaluationResult(query='In the essay, the author mentions his early experiences with programming. Describe the first computer he used for programming, the language he used, and the challenges he faced.', contexts=["I remember vividly how impressed and envious I felt watching him sitting in front of it, typing programs right into the computer.\n\nComputers were expensive in those days and it took me years of nagging before I convinced my father to buy one, a TRS-80, in about 1980. The gold standard then was the Apple II, but a TRS-80 was good enough. This was when I really started programming. I wrote simple games, a program to predict how high my model rockets would fly, and a word processor that my father used to write at least one book. There was only room in memory for about 2 pages of text, so he'd write 2 pages at a time and then print them out, but it was a lot better than a typewriter.\n\nThough I liked programming, I didn't plan to study it in college. In college I was going t

In [92]:
print(eval_results["correctness"][:1])

[EvaluationResult(query='In the essay, the author mentions his early experiences with programming. Describe the first computer he used for programming, the language he used, and the challenges he faced.', contexts=None, response="The author's first computer for programming was the IBM 1401, which was located in the basement of his junior high school. The language he used was an early version of Fortran. One of the challenges he faced was the limited input options for programs, as the only form of input was data stored on punched cards, which he did not have. This limitation led him to struggle with finding meaningful tasks to perform with the computer.", passing=True, feedback="The generated answer is highly relevant and almost completely accurate. It correctly identifies the first computer the author used (IBM 1401), the programming language (Fortran), and the challenges he faced (limited input options). However, it lacks some details present in the reference answer, such as the autho

In [93]:
print(eval_results["guideline"][:1])

[EvaluationResult(query='In the essay, the author mentions his early experiences with programming. Describe the first computer he used for programming, the language he used, and the challenges he faced.', contexts=None, response="The author's first computer for programming was the IBM 1401, which was located in the basement of his junior high school. The language he used was an early version of Fortran. One of the challenges he faced was the limited input options for programs, as the only form of input was data stored on punched cards, which he did not have. This limitation led him to struggle with finding meaningful tasks to perform with the computer.", passing=True, feedback="The response fully answers the query by providing detailed information about the author's first computer for programming, the language he used, and the challenges he faced. It is well-structured and clear.", score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)]


In [94]:
print(eval_results["semantic"][:1])

[EvaluationResult(query=None, contexts=None, response=None, passing=True, feedback='Similarity score: 0.9724231865833315', score=0.9724231865833315, pairwise_source=None, invalid_result=False, invalid_reason=None)]
